In [1]:
#1. 이미지(28*28*3) -> conv1 -> conv2 -> 신경망 -> softmax(0~9)
#2. 신경망(dropout, softmax) -> 0~9 분류 결과

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()# 실행시마다 그래프를 다시 생성
np.random.seed(20190618)
input_data.read_data_sets("/tmp/data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001E4539C9C18>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001E453A701D0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001E453A70780>)

In [19]:
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [23]:
#첫번째 컨볼루셔널 계층 생성
#이미지(28,28,1)
#필터 (5,5,1)를 32개 적용 -> 총 32개의 특징이 추출됨
num_filters1=32 #32개의 필터
x=tf.placeholder(tf.float32, [None, 784])
ximage=tf.reshape(x, [-1,28,28,1]) #28*28의 흑백이미지, 이미지 개수는 모름(-1,None)
wconv1=tf.Variable(tf.random_normal([5,5,1,num_filters1])) #필터 크기
hconv1=tf.nn.conv2d(ximage, wconv1, strides=[1,1,1,1], padding='SAME')
#ximage이미지에 대해 wconv1이라는 필터를 적용하는데, 오른쪽 1칸, 아래쪽 1줄씩 이동해가면서 합성곱을 연산해라. [1, 오른쪽, 아래쪽, 1]
bconv1=tf.Variable(tf.random_normal([num_filters1]))
hconv1_cutoff=tf.nn.relu(hconv1+bconv1)
hpool1=tf.nn.max_pool(hconv1_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
#커널크기: 2*2. 이동은 우측:2칸, 아래쪽:2줄 

#두번째 컨볼루셔널 계층 생성
num_filters2=64 #64개의 필터
wconv2=tf.Variable(tf.random_normal([5,5,num_filters1,num_filters2])) #필터 크기
hconv2=tf.nn.conv2d(hpool1, wconv2, strides=[1,1,1,1], padding='SAME')
#ximage이미지에 대해 wconv1이라는 필터를 적용하는데, 오른쪽 1칸, 아래쪽 1줄씩 이동해가면서 합성곱을 연산해라. [1, 오른쪽, 아래쪽, 1]
bconv2=tf.Variable(tf.random_normal([num_filters2]))
hconv2_cutoff=tf.nn.relu(hconv2+bconv2)
hpool2=tf.nn.max_pool(hconv2_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
#커널크기: 2*2. 이동은 우측:2칸, 아래쪽:2줄 
#(14,14) => (7,7)

#완전연결계층 구성(fully connectred layer)
hpool2flat=tf.reshape(hpool2, [-1, 7*7*num_filters2]) #64개의 7*7 행렬을 1차원으로 변환
num_units1=7*7*num_filters2
num_units2=1024 #1024개의 뉴런으로 구성
w2=tf.Variable(tf.random_normal([num_units1, num_units2]))
b2=tf.Variable(tf.random_normal([num_units2]))
hidden2 = tf.nn.relu(tf.matmul(hpool2flat, w2)+b2)
keep_prob=tf.placeholder(tf.float32)
hidden2_drop=tf.nn.dropout(hidden2, keep_prob)

w0=tf.Variable(tf.zeros([num_units2, 10]))
b0=tf.Variable(tf.zeros([10]))
k=tf.matmul(hidden2_drop, w0)+b0
p=tf.nn.softmax(k)

#비용(cost)함수
t=tf.placeholder(tf.float32, [None,10])
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(k,t))
train=tf.train.AdamOptimizer(0.0001).minimize(cost)
correct_prediction=tf.equal(tf.argmax(p,1), tf.argmax(t,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#세션생성
sess=tf.Session()
sess.run(tf.global_variables_initializer())

#training
for step in range(10000):
    batch_xs, batch_ys =mnist.train.next_batch(50)
    sess.run(train, feed_dict={x:batch_xs, t:batch_ys, keep_prob:0.5})
    if step % 500 == 0:
        costvs, accvs=[],[]
        for c in range(4):
            start = int(len(mnist.test.labels)/4*c)
            end = int(len(mnist.test.labels)/4*(c+1))
            costv, accv = sess.run([cost, accuracy], 
                                  feed_dict={x:mnist.test.images[start:end],
                                            t:mnist.test.labels[start:end],
                                            keep_prob:1.0})
            costvs.append(costv)
            accvs.append(accv)
        cost_val=np.sum(costvs)
        acc_val=np.sum(accvs)
        print("step:%d, cost:%f, acc:%f"%(step, cost_val, acc_val))

step:0, cost:-8233.249023, acc:0.395600


KeyboardInterrupt: 